## Implementation using Felipe's final formula

In [ ]:
#def Mforecast3(tipo, process, param,i,T):
#        M = []
#        #C_NT_0 = 1
        #print(param)
#        t0 = i
#        tau_mu = param[0]
#        tau_NT = param[1]
#        if tipo == "mark":
#            if process == "MU":
#                alpha_appo = np.exp(-1/param[0])
#                C_appo = np.fromfunction(lambda i,j : alpha_appo**np.abs(i-j), (T+1,T+1), dtype = float)
#            if process == "NT":
#                alpha_appo = np.exp(-1/param[1])
#                C_appo = np.fromfunction(lambda i,j : alpha_appo**np.abs(i-j), (T,T), dtype = float)
#        if tipo == "osc":
#            if process == "MU":
#                alpha_mu = np.exp(-1/param[0])
#                tau1_mu = param[1]
#                C_appo = np.fromfunction(lambda i,j : alpha_mu**np.abs(i-j)*np.sin((i-j)/tau1_mu+np.pi/2), (T+1,T+1), dtype = float)
#            if process == "NT":
#                alpha_NT = np.exp(-1/param[2])
#                tau1_NT = param[3]
#                C_appo = np.fromfunction(lambda i,j : alpha_NT**np.abs(i-j)*np.sin((i-j)/(tau1_NT)+np.pi/2), (T+1,T+1), dtype = float)
#                
#                
#        if tipo == "id":
#            if process == "MU":
#                alpha_appo = np.exp(-1/param[0])
#                C_appo = np.identity(T)
#            if process == "NT":
#                alpha_appo = np.exp(-1/(param[0]))
#                C_appo = np.fromfunction(lambda i,j : alpha_appo**np.abs(i-j)*np.sin((i-j)/(tau_NT)+np.pi/2), (T+1,T+1), dtype = float)
#        
#        if tipo == "pl":
#            if process == "MU":
#                tau = param[0]
#                gamma = param[1]
#                C_appo = (np.fromfunction(lambda i, j: np.power( 1+np.absolute(i-j)/tau,-gamma), (T+1,T+1), dtype=float))
#            if process == "NT":
#                tau = param[2]
#                gamma = param[3]
#                C_appo = (np.fromfunction(lambda i, j: np.power( 1+np.absolute(i-j)/tau,-gamma), (T+1,T+1), dtype=float))
#        
#        g = np.zeros((T-t0+1,t0))
#        for J in range (t0,T+1):
#            C = C_appo[:J+1,:J+1]
#            inv_C = np.linalg.inv(C)
#            inv_C_int = inv_C[t0:J,t0:J]
#            inv_inv_C_int = np.linalg.inv(inv_C_int)
#            #delta_inv_C = +inv_C[:,I:J-1].dot(inv_inv_C_int).dot(inv_C[I:J-1,:])
#            W = C[t0:J,J]
#            forecast_line = ((((W.T).dot(inv_inv_C_int)).dot(W) - inv_C[J,J])**(-1))*(C[J,:t0] - ((W.T).dot(inv_inv_C_int)).dot(inv_C[t0:J,:t0]))
#            g[J-t0-1,:t0] = forecast_line
#
#        return g

# A simple decorator for measuring perfomance/time of execution

In [3]:
from time import time
from functools import wraps

def simple_time_tracker(log_fun):
    def _simple_time_tracker(fn):
        @wraps(fn)
        def wrapped_fn(*args, **kwargs):
            start_time = time()

            try:
                result = fn(*args, **kwargs)
            finally:
                elapsed_time = time() - start_time

                # log the result
                log_fun({
                    'function_name': fn.__name__,
                    'total_time': elapsed_time,
                })
                
            return result

        return wrapped_fn
    return _simple_time_tracker


def _log(message):
    print('[SimpleTimeTracker] {function_name} {total_time:.3f}'.format(**message))


# Original Michele's function

In [9]:
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt
import matplotlib
from scipy import linalg
from mpmath import *
from timeit import default_timer as timer


#@simple_time_tracker(_log)
def Mforecast(tipo, process, param,i,T):
        M = []
        #C_NT_0 = 1
        #print(param)
        t0 = i
        tau_mu = param[0]
        tau_NT = param[1]
        if tipo == "mark":
            if process == "MU":
                alpha_appo = np.exp(-1/param[0])
                C_appo = np.fromfunction(lambda i,j : alpha_appo**np.abs(i-j), (T,T), dtype = float)
            if process == "NT":
                alpha_appo = np.exp(-1/param[1])
                C_appo = np.fromfunction(lambda i,j : alpha_appo**np.abs(i-j), (T,T), dtype = float)
        if tipo == "osc":
            if process == "MU":
                alpha_mu = np.exp(-1/param[0])
                tau1_mu = param[1]
                C_appo = np.fromfunction(lambda i,j : alpha_mu**np.abs(i-j)*np.sin((i-j)/tau1_mu+np.pi/2), (T,T), dtype = float)
            if process == "NT":
                alpha_NT = np.exp(-1/param[2])
                tau1_NT = param[3]
                C_appo = np.fromfunction(lambda i,j : alpha_NT**np.abs(i-j)*np.sin((i-j)/(tau1_NT)+np.pi/2), (T,T), dtype = float)
                
                
        if tipo == "id":
            if process == "MU":
                alpha_appo = np.exp(-1/param[0])
                C_appo = np.identity(T)
            if process == "NT":
                alpha_appo = np.exp(-1/(param[0]))
                C_appo = np.fromfunction(lambda i,j : alpha_appo**np.abs(i-j)*np.sin((i-j)/(tau_NT)+np.pi/2), (T,T), dtype = float)
        
        if tipo == "pl":
            if process == "MU":
                tau = param[0]
                gamma = param[1]
                C_appo = (np.fromfunction(lambda i, j: np.power( 1+np.absolute(i-j)/tau,-gamma), (T,T), dtype=float))
            if process == "NT":
                tau = param[2]
                gamma = param[3]
                C_appo = (np.fromfunction(lambda i, j: np.power( 1+np.absolute(i-j)/tau,-gamma), (T,T), dtype=float))
                
        #print(C_appo)
        
        g = np.zeros((T-t0,t0))
        for I in range (t0,t0+1):
            for J in range (I+1,T+1):
                C = C_appo[:J,:J]
                inv_C = np.linalg.inv(C)
                inv_C_int = inv_C[I:J-1,I:J-1]
                inv_inv_C_int = np.linalg.inv(inv_C_int)
                delta_inv_C = +inv_C[:,I:J-1].dot(inv_inv_C_int).dot(inv_C[I:J-1,:])
                matrix = (inv_C[J-1,J-1]-delta_inv_C[J-1,J-1])**(-1)*(inv_C-delta_inv_C)
                M.append(matrix)
                g[J-t0-1,:t0] = M[0][J-1,:t0]
                M = []
            M = []
        return g

# Profiling the function - where is the problem ?

In [15]:
from line_profiler import LineProfiler

In [16]:
lp = LineProfiler()
lp_wrapper = lp(Mforecast)
lp_wrapper("mark", "MU", [1,1], 10,100)
lp.print_stats()

Timer unit: 1e-06 s

Total time: 0.034305 s
File: <ipython-input-12-3047e4ed96f4>
Function: Mforecast at line 17

Line #      Hits         Time  Per Hit   % Time  Line Contents
    17                                           def Mforecast(tipo, process, param,i,T):
    18         1        251.0    251.0      0.7          M = []
    19                                                   #C_NT_0 = 1
    20                                                   #print(param)
    21         1          2.0      2.0      0.0          t0 = i
    22         1          2.0      2.0      0.0          tau_mu = param[0]
    23         1          2.0      2.0      0.0          tau_NT = param[1]
    24         1          2.0      2.0      0.0          if tipo == "mark":
    25         1          1.0      1.0      0.0              if process == "MU":
    26         1         22.0     22.0      0.1                  alpha_appo = np.exp(-1/param[0])
    27         1        637.0    637.0      1.9             

**Conclusion: The problem is mainly the MATRIX INVERSION.**

# Optimized function - Numba + Truncation of very small numbers

In [27]:
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt
import matplotlib
from scipy import linalg
from mpmath import *

def Mforecast_Optimized(tipo, process, param, t0, T):

        tau_mu = param[0]
        tau_NT = param[1]
        if tipo == "mark":
            if process == "MU":
                alpha_appo = np.exp(-1/param[0])
                C_appo = np.fromfunction(lambda i,j : alpha_appo**np.abs(i-j), (T,T), dtype = float)
            if process == "NT":
                alpha_appo = np.exp(-1/param[1])
                C_appo = np.fromfunction(lambda i,j : alpha_appo**np.abs(i-j), (T,T), dtype = float)
        if tipo == "osc":
            if process == "MU":
                alpha_mu = np.exp(-1/param[0])
                tau1_mu = param[1]
                C_appo = np.fromfunction(lambda i,j : alpha_mu**np.abs(i-j)*np.sin((i-j)/tau1_mu+np.pi/2), (T,T), dtype = float)
            if process == "NT":
                alpha_NT = np.exp(-1/param[2])
                tau1_NT = param[3]
                C_appo = np.fromfunction(lambda i,j : alpha_NT**np.abs(i-j)*np.sin((i-j)/(tau1_NT)+np.pi/2), (T,T), dtype = float)
                     
        if tipo == "id":
            if process == "MU":
                alpha_appo = np.exp(-1/param[0])
                C_appo = np.identity(T)
            if process == "NT":
                alpha_appo = np.exp(-1/(param[0]))
                C_appo = np.fromfunction(lambda i,j : alpha_appo**np.abs(i-j)*np.sin((i-j)/(tau_NT)+np.pi/2), (T,T), dtype = float)
        
        if tipo == "pl":
            if process == "MU":
                tau = param[0]
                gamma = param[1]
                C_appo = (np.fromfunction(lambda i, j: np.power( 1+np.absolute(i-j)/tau,-gamma), (T,T), dtype=float))
            if process == "NT":
                tau = param[2]
                gamma = param[3]
                C_appo = (np.fromfunction(lambda i, j: np.power( 1+np.absolute(i-j)/tau,-gamma), (T,T), dtype=float))
    
        g = np.zeros((T-t0,t0))
        for J in range (t0+1,T+1):
            C = C_appo[:J,:J]
            inv_C = optimized_matrix_inverse(C)
            inv_C[np.abs(inv_C) < pow(10,-18)] = 0 ###added line
            inv_C_int = inv_C[t0:J-1,t0:J-1]
            inv_inv_C_int = optimized_matrix_inverse(inv_C_int)
            inv_inv_C_int[np.abs(inv_inv_C_int) < pow(10,-18)] = 0 ###added line
            delta_inv_C = +inv_C[:,t0:J-1].dot(inv_inv_C_int).dot(inv_C[t0:J-1,:])
            delta_inv_C[np.abs(delta_inv_C) < pow(10,-18)] = 0 ##added line
            matrix = (inv_C[J-1,J-1]-delta_inv_C[J-1,J-1])**(-1)*(inv_C-delta_inv_C)
            g[J-t0-1,:t0] = matrix[J-1,:t0]
        return g

**Original function performance in a (990,10) Forecast Matrix**

In [28]:
%timeit np.flip(Mforecast("mark", "MU", [1,1], 10,1000),1)

4min 40s ± 2 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


**Performance with Numba + Truncation of small numbers**

In [29]:
import numpy as np
from numba import njit

@njit
def optimized_matrix_inverse(matrix):
    return np.linalg.inv(matrix)

In [30]:
%timeit np.flip(Mforecast_Optimized("mark", "MU", [1,1], 10,1000),1)

1min 58s ± 1.28 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


**Conclusion: Truncation can improve significantly the performance of the inversion operation!**

**Perfomance with Cython**

In [34]:
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [35]:
%%cython -a

import numpy as np
def optimized_matrix_inverse(matrix):
    return np.linalg.inv(matrix)

In [28]:
%timeit Mforecast2("mark", "MU", [1,1], 10,600)

16.1 s ± 965 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


**Conclusion: only Cython does not solve the problem!**

## <span style='color:red' > Trying to invert the correlation matrix using other strategies... </span>

## Trying to solve using a linear system approach

**We can try to use the following approach:**

Solve the linear system $\mathsf{Cx = I}$, to compute the inverse matrix $x = C^{-1}$, using <em>scipy.sparse.linalg.spsolve(A, b, permc_spec=None, use_umfpack=True)</em>.

In [1]:
import numpy as np
T = 500
C_appo = np.zeros((T,T))
alpha_appo = float(np.exp(-1))
for i in range(0,T):
    for j in range(0,T):
        C_appo[i,j] = float(alpha_appo**np.abs(i-j))

In [2]:
type(C_appo)

numpy.ndarray

In [3]:
from scipy.sparse.linalg import spsolve
import numpy as np

def inverse_matrix_linear_system_approach(matrix):
    T = matrix.shape[0]
    return spsolve(matrix,np.identity(T))

**Perfomance with the Linear System Approach**

In [4]:
%timeit inverse_matrix_linear_system_approach(C_appo)

/opt/anaconda3/lib/python3.7/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:138: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)


137 ms ± 2.51 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


**Performance using only numpy**

In [5]:
%timeit np.linalg.inv(C_appo)

14.4 ms ± 396 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


**Conclusion: The linear system apporach is not efficient!**

## Using Cramer's rule

In [6]:
def transposeMatrix(m):
    return list(map(list,zip(*m)))

def getMatrixMinor(m,i,j):
    return [row[:j] + row[j+1:] for row in (m[:i]+m[i+1:])]

def getMatrixDeternminant(m):
    #base case for 2x2 matrix
    if len(m) == 2:
        return m[0][0]*m[1][1]-m[0][1]*m[1][0]

    determinant = 0
    for c in range(len(m)):
        determinant += ((-1)**c)*m[0][c]*getMatrixDeternminant(getMatrixMinor(m,0,c))
    return determinant

def getMatrixInverse(m):
    determinant = getMatrixDeternminant(m)
    #special case for 2x2 matrix:
    if len(m) == 2:
        return [[m[1][1]/determinant, -1*m[0][1]/determinant],
                [-1*m[1][0]/determinant, m[0][0]/determinant]]

    #find matrix of cofactors
    cofactors = []
    for r in range(len(m)):
        cofactorRow = []
        for c in range(len(m)):
            minor = getMatrixMinor(m,r,c)
            cofactorRow.append(((-1)**(r+c)) * getMatrixDeternminant(minor))
        cofactors.append(cofactorRow)
    cofactors = transposeMatrix(cofactors)
    for r in range(len(cofactors)):
        for c in range(len(cofactors)):
            cofactors[r][c] = cofactors[r][c]/determinant
    return cofactors

In [7]:
C_inv = getMatrixInverse(C_appo.tolist())

KeyboardInterrupt: 

**Not finished in "finite" time for T = 500 - Reason: Cramer's rule for evaluating determinantes is O(N!).**

## Gauss Elimination Method

In [8]:
%load_ext Cython

In [9]:
%%cython -a
def eliminate(r1, r2, col, target=0):
    fac = (r2[col]-target) / r1[col]
    for i in range(len(r2)):
        r2[i] -= fac * r1[i]

def gauss(a):
    for i in range(len(a)):
        if a[i][i] == 0:
            for j in range(i+1, len(a)):
                if a[i][j] != 0:
                    a[i], a[j] = a[j], a[i]
                    break
            else:
                raise ValueError("Matrix is not invertible")
        for j in range(i+1, len(a)):
            eliminate(a[i], a[j], i)
    for i in range(len(a)-1, -1, -1):
        for j in range(i-1, -1, -1):
            eliminate(a[i], a[j], i)
    for i in range(len(a)):
        eliminate(a[i], a[i], i, target=1)
    return a


def inverse(a):
    tmp = [[] for _ in a]
    for i,row in enumerate(a):
        assert len(row) == len(a)
        tmp[i].extend(row + [0]*i + [1] + [0]*(len(a)-i-1))
    gauss(tmp)
    ret = []
    for i in range(len(tmp)):
        ret.append(tmp[i][len(tmp[i])//2:])
    return ret

In [10]:
%timeit np.array(inverse(C_appo.tolist()))

7.04 s ± 131 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


**Conclusion: too much slower than np.inv!** \
**PS: Not possible using Numba due to some kind of not fixed item size in lists.**